In [1]:
import numpy as np
import scipy.stats
import scipy.special

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
%matplotlib inline

import gensim
#from keras.preprocessing import text
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import re
import nltk
import json

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats.stats import pearsonr


from sklearn import decomposition, datasets
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from IPython.display import display, HTML
from operator import itemgetter
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Add, Dot, Flatten
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [4]:
district_data=pd.read_csv("/content/Tea in Tamil Nadu.csv")

In [7]:
district_data.dropna(how='any',axis=0)

,Sl. No.,District,AREA,PRODN.,PTY.
1,2,Coimbatore,11186.0,150340.0,13.00
4,5,Dindigul,3.0,40.0,13.44
7,8,Kanyakumari,202.0,2715.0,13.44
19,20,Theni,1676.0,22525.0,13.00
20,21,The Nilgiris,55779.0,749670.0,13.44
27,28,Tirunelveli,804.0,10806.0,13.44
31,Total,State Total,69650.0,936096.0,13.44


In [8]:
alternate_data=pd.read_csv("/content/kenya-monthly-tea-production-2012-2016.csv")
alternate_data.head()

,Quantity Produced (Kgs),Year,Month
0,"36,205,286",2012,JANUARY
1,"18,411,613",2012,FEBRUARY
2,"17,858,709",2012,MARCH
3,"18,118,143",2012,APRIL
4,"37,383,207",2012,MAY


In [12]:
tea_nepal=pd.read_csv("/content/tea.csv")
tea_nepal.head()

,DISTRICT,AREA,PROD.
0,Jhapa,11557,18732906
1,Ilam,8045,4933682
2,Panchathar,1352,508063
3,Dhankuta,961,184721


In [13]:
tea_export=pd.read_csv("/content/export_dt.csv")

In [14]:
tea_export

,date,TEA_KOLKATA,OIL
0,2017-09-01,2.610435,52.950000
1,2017-08-01,2.758033,49.943333
2,2017-07-01,2.887943,47.656667
3,2017-06-01,2.839322,46.166667
4,2017-05-01,2.674872,49.893333
...,...,...,...
688,1960-05-01,1.121401,1.630000
689,1960-04-01,1.121401,1.630000
690,1960-03-01,1.121401,1.630000
691,1960-02-01,1.121401,1.630000


In [18]:
crop=pd.read_csv("/content/crop_production.csv")
crop=crop[crop.Crop.isin(['Tea','Coffee'])]

In [28]:
produce=pd.read_csv("/content/produce.csv")
produce=produce.drop(axis=1,columns=produce.columns[3:15])

In [40]:
produce.head()

,Particulars,Frequency,Unit,3-2005,3-2006,3-2007,3-2008,3-2009,3-2010,3-2011,3-2012,3-2013,3-2014
0,Agricultural Production Foodgrains,"Annual, Ending mar Of Each Year",Ton mn,198.36282,208.6016,217.28212,230.77504,234.466177,218.107372,244.491809,259.286253,257.13,264.3806
1,Agricultural Production Foodgrains Kharif,"Annual, Ending mar Of Each Year",Ton mn,103.30942,109.8734,110.57622,120.95724,118.138577,103.954032,120.853191,131.274812,128.07,129.3686
2,Agricultural Production Foodgrains Rabi,"Annual, Ending mar Of Each Year",Ton mn,95.05340,98.7282,106.70590,109.81780,116.327600,114.153340,123.638618,128.011441,129.06,135.0120
3,Agricultural Production Foodgrains Rice,"Annual, Ending mar Of Each Year",Ton mn,83.13170,91.7934,93.35530,96.69290,99.182500,89.092901,95.979819,105.301100,105.24,106.2872
4,Agricultural Production Foodgrains Rice Kharif,"Annual, Ending mar Of Each Year",Ton mn,72.23000,78.2719,80.17080,82.65940,84.908200,75.916510,80.651241,92.780700,92.37,92.0093


In [38]:
datafile=pd.read_csv("/content/datafile (1).csv")
others=datafile[datafile.State.isin(['Tamil Nadu','Andhra Pradesh'])]

In [34]:
datafile.State.unique()

array(['Uttar Pradesh', 'Karnataka', 'Gujarat', 'Andhra Pradesh',
       'Maharashtra', 'Punjab', 'Haryana', 'Rajasthan', 'Madhya Pradesh',
       'Tamil Nadu', 'Bihar', 'Orissa', 'West Bengal'], dtype=object)

In [41]:
croptypes=pd.read_csv("/content/datafile (2).csv")
croptypes

,Crop,Production 2006-07,Production 2007-08,Production 2008-09,Production 2009-10,Production 2010-11,Area 2006-07,Area 2007-08,Area 2008-09,Area 2009-10,Area 2010-11,Yield 2006-07,Yield 2007-08,Yield 2008-09,Yield 2009-10,Yield 2010-11
0,Total Foodgrains,158.8,168.6,171.3,159.4,178.9,128.5,128.8,127.6,126.0,131.7,123.6,130.9,134.3,126.5,135.9
1,Rice,200.8,207.9,213.3,191.6,206.4,168.5,168.9,175.1,161.2,164.8,119.2,123.1,121.8,118.9,125.2
2,Wheat,131.6,136.4,140.1,140.3,150.8,115.0,115.2,114.0,116.9,119.5,114.4,118.4,122.8,120.0,126.3
3,Jowar,124.3,137.8,126.0,116.5,121.8,120.7,110.6,107.3,111.0,105.2,103.0,124.6,117.4,105.0,115.8
4,Bajra,136.4,161.5,143.9,105.4,167.9,94.5,95.1,87.0,88.5,95.6,144.3,169.7,165.4,119.0,175.8
5,Maize,198.8,249.6,259.8,220.1,286.0,156.1,160.5,161.6,163.4,169.1,127.3,155.5,160.7,134.7,169.1
6,Ragi,56.1,83.6,79.3,73.4,85.2,60.2,70.9,70.6,64.8,65.8,93.2,117.9,112.2,113.2,129.6
7,Small millets,53.9,61.9,50.0,42.9,82.0,50.9,52.3,45.6,41.8,45.0,106.0,118.3,109.7,102.7,182.4
8,Barley,88.1,79.3,112.0,89.9,110.3,72.8,67.9,79.5,70.2,79.4,121.0,116.9,141.0,127.9,138.9
9,Coarse Cereals,138.5,166.4,163.5,137.0,178.4,106.4,105.6,101.8,102.6,105.4,130.1,157.6,160.7,133.5,169.2


In [45]:
d=pd.read_csv("/content/MEPm.csv")
temp=d[['Tmax','Tmin','W']]